In [0]:
!pip install h5py pyyaml
!pip install tf_nightly

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.15.0-dev20190730'

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [0]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()

W0731 09:38:55.284304 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0731 09:38:55.285528 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0731 09:38:55.288083 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0731 09:38:55.289984 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0731 09:38:55.293409 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0731 09:38:55.296646 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0731 09:38:55.299530 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0731 09:38:55.302395 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

W0731 09:39:24.741672 140593696061312 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:468: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2477 - acc: 0.6466
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 623us/sample - loss: 1.1490 - acc: 0.6730 - val_loss: 0.7145 - val_acc: 0.7810
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4191 - acc: 0.8839
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 285us/sample - loss: 0.4112 - acc: 0.8830 - val_loss: 0.5809 - val_acc: 0.8120
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2750 - acc: 0.9310
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 276us/sample - loss: 0.2830 - acc: 0.9270 - val_loss: 0.4432 - val_acc: 0.8620
Epoch 4/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2018 - acc: 0.9554
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [0]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [0]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 132us/sample - loss: 2.3916 - acc: 0.0780
Untrained model, accuracy:  7.80%


In [0]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 76us/sample - loss: 0.4231 - acc: 0.8580
Restored model, accuracy: 85.80%


In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 273us/sample - loss: 1.1366 - acc: 0.6860
Epoch 2/5
1000/1000 [==============================] - 0s 179us/sample - loss: 0.4271 - acc: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 179us/sample - loss: 0.2907 - acc: 0.9180
Epoch 4/5
1000/1000 [==============================] - 0s 178us/sample - loss: 0.2288 - acc: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 173us/sample - loss: 0.1623 - acc: 0.9610


In [0]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0731 09:58:12.904221 140593696061312 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 09:58:12.907056 140593696061312 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 149us/sample - loss: 0.4312 - acc: 0.8570
Restored model, accuracy: 85.70%


In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

W0731 09:59:13.549267 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0731 09:59:13.550845 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0731 09:59:13.552720 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0731 09:59:13.557319 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0731 09:59:13.558132 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0731 09:59:13.559848 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0731 09:59:13.561800 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0731 09:59:13.563588 140593696061312 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 277us/sample - loss: 1.1125 - acc: 0.6950
Epoch 2/5
1000/1000 [==============================] - 0s 161us/sample - loss: 0.4393 - acc: 0.8730
Epoch 3/5
1000/1000 [==============================] - 0s 181us/sample - loss: 0.2918 - acc: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 172us/sample - loss: 0.2093 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 183us/sample - loss: 0.1486 - acc: 0.9640
